## Generate Subjective Value (SV) difference

Use the BHM generated parameters to generate SV difference

Wrote this script to meet a deadline for SFN conference but in the future we want this to be incorporated into the pipeline.


### Import libraries and modules

In [8]:
import pandas as pd
import os,sys
pd.options.display.max_rows = 999
pd.options.display.max_columns = 99

In [9]:
# getting the name of the directory
# where the this file is present.
# current = os.path.dirname(os.path.realpath(__file__))
current = os.path.dirname(os.getcwd())
 
# Getting the parent directory name
# where the current directory is present.
# parent = os.path.dirname(os.path.dirname(current))
# parent = current
parent = '/Users/pizarror/IDM'
# print(parent)
#/Users/pizarror/IDM

# adding the parent directory to
# the sys.path.
sys.path.append(parent)

from IDM_model.src import model_functions as mf


In [10]:
# these functions do not mention either task

def get_df_bhm(idm_dir=''):
    agg_fn = os.path.join(idm_dir,'Aggregate_All_Model_Parameters.csv')
    df_agg = pd.read_csv(agg_fn)
    cols = [c for c in list(df_agg) if 'bhm' in c or 'subject' in c]
    df_bhm = df_agg.loc[:,cols]
    return df_bhm


In [11]:
# these function is generalized to work for both CRDM and CDD

def read_load_data(subject='23_IDM_0144',fn='/tmp',alpha0=1.0,cols=[],task='crdm'):
    df = pd.read_csv(fn)
    if 'crdm' in task:
        domain = 'gain'
        df = mf.get_by_domain(df,domain=domain,task=task,verbose=True)
    df = mf.drop_pract(df,task=task)
    df,response_rate = mf.drop_non_responses(df,task=task,verbose=False) 
    conf_resp = mf.get_confresp(df,task=task)
    cols += [conf_resp]
    data = mf.get_data(df,cols,task=task)[0]
    data = data.rename(columns={conf_resp:'{}_confkey'.format(task)})
    data['subject'] = subject
    return data 

def get_SV_hat_fn(sfn,task='crdm'):
    SV_fn = sfn.replace('split','utility')
    SV_dir = os.path.dirname(SV_fn)
    if 'cdd_nlh' in task:
        SV_dir = SV_dir.replace('cdd',task)
    if 'crdm' in task:
        SV_fn = os.path.join(SV_dir,'bh',os.path.basename(SV_fn).replace('.csv','_gain_SV_hat_bhm.csv'))
    else:
        SV_fn = os.path.join(SV_dir,'bh',os.path.basename(SV_fn).replace('.csv','_SV_hat_bhm.csv'))
    return SV_fn


In [12]:
# these functions are specific to crdm

def get_crdm_params(df_bhm,subject):
    alpha = df_bhm.loc[df_bhm['subject']==subject,'crdm_alpha_bhm'].values
    beta = df_bhm.loc[df_bhm['subject']==subject,'crdm_beta_bhm'].values
    return alpha,beta

def get_crdm_trials(df):
    vl = df['crdm_lott_amt'].values
    pl = df['crdm_lott_p'].values
    a = df['crdm_amb_lev'].values
    vs = df['crdm_sure_amt'].values
    ps = df['crdm_sure_p'].values
    return vl,pl,a,vs,ps

def append_crdm_SV_delta(df_bhm,df):
    subject = df['subject'].unique().item()
    choice_col = mf.get_choicecol(df,task='crdm')
    conf_resp = mf.get_confresp(df,task='crdm')

    vl,pl,a,vs,ps = get_crdm_trials(df)
    alpha,beta = get_crdm_params(df_bhm,subject)

    df['SV_lottery'] = (pl - beta*a/2) * (vl**alpha)
    df['SV_safe'] = (ps) * (vs**alpha)
    df['SV_delta'] = df['SV_lottery'] - df['SV_safe']
    df['valence'] = 2.0*df[choice_col] - 1.0
    df['confidence'] = df[conf_resp]*df['valence']
    df['ambig_trial'] = 0
    df['ambig_trial'] = df['crdm_amb_lev'] > 0
    
    return df.loc[:,['SV_delta','ambig_trial','confidence']].astype(float)

def crdm_SV_delta_save(split_dir,df_bhm):
    task = 'crdm'
    for s in df_bhm['subject']:
        cols = ['crdm_choice','crdm_sure_p','crdm_lott_p','crdm_sure_amt','crdm_lott_amt','crdm_amb_lev']
        sfn = os.path.join(split_dir,s,task,'{}_{}.csv'.format(s,task))
        subj_data = read_load_data(subject=s,fn=sfn,alpha0=1.0,cols=cols,task=task)
        for c in list(subj_data):
            if 'subject' in c:
                continue
            subj_data[c] = subj_data[c].astype(float)
        df_SV_hat = append_crdm_SV_delta(df_bhm,subj_data)
        SV_fn = get_SV_hat_fn(sfn,task=task)
        print('Saving SV_hat to : {}'.format(SV_fn))
        df_SV_hat.to_csv(SV_fn,index=False)


In [13]:
# these functions are specific to cdd

def get_cdd_params(df_bhm,subject):
    kappa = df_bhm.loc[df_bhm['subject']==subject,'cdd_kappa_bhm'].values
    return kappa

def get_cdd_trials(df):
    da = df['cdd_delay_amt'].values
    dw = df['cdd_delay_wait'].values
    ia = df['cdd_immed_amt'].values
    iw = df['cdd_immed_wait'].values
    return da,dw,ia,iw

def append_cdd_SV_delta(df_bhm,df):
    subject = df['subject'].unique().item()
    choice_col = mf.get_choicecol(df,task='cdd')
    conf_resp = mf.get_confresp(df,task='cdd')

    da,dw,ia,iw = get_cdd_trials(df)
    kappa = get_cdd_params(df_bhm,subject)

    df['SV_delay'] = da / (1+kappa*dw)
    df['SV_immediate'] = ia / (1+kappa*iw)
    df['SV_delta'] = df['SV_delay'] - df['SV_immediate']
    df['valence'] = 2.0*df[choice_col] - 1.0
    df['confidence'] = df[conf_resp]*df['valence']
    df['ambig_trial'] = 0
    
    return df.loc[:,['SV_delta','ambig_trial','confidence']].astype(float)

def cdd_SV_delta_save(split_dir,df_bhm):
    task = 'cdd'
    for s in df_bhm['subject']:
        cols = ['cdd_choice','cdd_delay_amt','cdd_delay_wait','cdd_immed_amt','cdd_immed_wait']
        sfn = os.path.join(split_dir,s,task,'{}_{}.csv'.format(s,task))
        subj_data = read_load_data(subject=s,fn=sfn,alpha0=1.0,cols=cols,task=task)
        for c in list(subj_data):
            if 'subject' in c:
                continue
            subj_data[c] = subj_data[c].astype(float)
        df_SV_hat = append_cdd_SV_delta(df_bhm,subj_data)
        SV_fn = get_SV_hat_fn(sfn,task=task)
        print('Saving SV_hat to : {}'.format(SV_fn))
        df_SV_hat.to_csv(SV_fn,index=False)


In [ ]:
# these functions are specific to cdd_nlh

def get_cdd_nlh_params(df_bhm,subject):
    alpha = df_bhm.loc[df_bhm['subject']==subject,'crdm_alpha_bhm'].values
    kappa = df_bhm.loc[df_bhm['subject']==subject,'cdd_nlh_kappa_bhm'].values
    return alpha,kappa

def append_cdd_nlh_SV_delta(df_bhm,df):
    subject = df['subject'].unique().item()
    choice_col = mf.get_choicecol(df,task='cdd')
    conf_resp = mf.get_confresp(df,task='cdd')

    da,dw,ia,iw = get_cdd_trials(df)
    alpha,kappa = get_cdd_nlh_params(df_bhm,subject)

    df['SV_delay'] = (da**alpha) / (1 + kappa*dw)
    df['SV_immediate'] = (ia**alpha) / (1 + kappa*iw)
    df['SV_delta'] = df['SV_delay'] - df['SV_immediate']
    df['valence'] = 2.0*df[choice_col] - 1.0
    df['confidence'] = df[conf_resp]*df['valence']
    df['ambig_trial'] = 0
    
    return df.loc[:,['SV_delta','ambig_trial','confidence']].astype(float)

def cdd_nlh_SV_delta_save(split_dir,df_bhm):
    task = 'cdd'
    for s in df_bhm['subject']:
        cols = ['cdd_choice','cdd_delay_amt','cdd_delay_wait','cdd_immed_amt','cdd_immed_wait']
        # experiment file from split dir is under cdd for cdd and cdd_nlh
        sfn = os.path.join(split_dir,s,task,'{}_{}.csv'.format(s,task))
        subj_data = read_load_data(subject=s,fn=sfn,alpha0=1.0,cols=cols,task=task)
        for c in list(subj_data):
            if 'subject' in c:
                continue
            subj_data[c] = subj_data[c].astype(float)
        df_SV_hat = append_cdd_nlh_SV_delta(df_bhm,subj_data)
        SV_fn = get_SV_hat_fn(sfn,task='cdd_nlh')
        print('Saving SV_hat to : {}'.format(SV_fn))
        df_SV_hat.to_csv(SV_fn,index=False)


In [14]:
idm_dir = '/Volumes/UCDN/datasets/IDM/'
df_bhm = get_df_bhm(idm_dir=idm_dir)
split_dir = os.path.join(idm_dir,'split')

# crdm_SV_delta_save(split_dir,df_bhm)
# cdd_SV_delta_save(split_dir,df_bhm)
cdd_nlh_SV_delta_save(split_dir,df_bhm)


Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0001/cdd/bh/23_IDM_0001_cdd_SV_hat_bhm.csv
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0002/cdd/bh/23_IDM_0002_cdd_SV_hat_bhm.csv
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0003/cdd/bh/23_IDM_0003_cdd_SV_hat_bhm.csv
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0004/cdd/bh/23_IDM_0004_cdd_SV_hat_bhm.csv
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0005/cdd/bh/23_IDM_0005_cdd_SV_hat_bhm.csv
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0006/cdd/bh/23_IDM_0006_cdd_SV_hat_bhm.csv
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0007/cdd/bh/23_IDM_0007_cdd_SV_hat_bhm.csv
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0008/cdd/bh/23_IDM_0008_cdd_SV_hat_bhm.csv
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0009/cdd/bh/23_IDM_0009_cdd_SV_hat_bhm.csv
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_001